In [1]:
# Exercise E.1

from functools import wraps

# 1) einfache Version: prüft alle positional Argumente
def positive(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        for value in args:
            if value <= 0:
                raise Exception("Sorry, not positive")
        return func(*args, **kwargs)
    return wrapper


@positive
def estimate_pi_dummy(N):
    return 4.0 * N / N


print(estimate_pi_dummy(10))
# print(estimate_pi_dummy(-5))  # -> Exception


# 2) erweiterte Version: prüft nur ein bestimmtes Argument
def positive_at(index=None):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            if index is None:
                values = args
            else:
                if index >= len(args):
                    raise Exception("Sorry, not positive")
                values = [args[index]]

            for value in values:
                if value <= 0:
                    raise Exception("Sorry, not positive")
            return func(*args, **kwargs)
        return wrapper
    return decorator


@positive_at(1)
def testfun(a, b, c):
    return a, b, c


# error:
# print(testfun(-1, -1, -1))

# kein error:
print(testfun(-1, 1, -1))


4.0
(-1, 1, -1)


In [2]:
# E.1 Test mit Ideen aus D.3 und D.4

@positive
def estimate_pi_mc(N: int) -> float:
    # vereinfachter Dummy für D.3-ähnliche Signatur
    return 4.0 * N / N


@positive_at(1)
def lv_step(x: float, dt: float, y: float):
    # D.4-ähnliche Funktion: nur dt muss positiv sein
    return x + dt * y


print("estimate_pi_mc(1000) =", estimate_pi_mc(1000))
# print(estimate_pi_mc(-10))   # -> Exception: Sorry, not positive

print("lv_step(10.0, 0.01, 2.0) =", lv_step(10.0, 0.01, 2.0))
# print(lv_step(10.0, -0.01, 2.0))  # -> Exception: Sorry, not positive


estimate_pi_mc(1000) = 4.0
lv_step(10.0, 0.01, 2.0) = 10.02


In [3]:
# Exercise E.2

%pip install pydantic

from pydantic import BaseModel, Field, ValidationError


# Beispiel 1: für estimate_pi(N)
class PiInput(BaseModel):
    N: int = Field(gt=0)  # > 0


def estimate_pi_checked(N: int) -> float:
    data = PiInput(N=N)  # validiert
    # hier nur Dummy-Rechnung
    return 4.0 * data.N / data.N


print(estimate_pi_checked(100))
# print(estimate_pi_checked(-5))  # ValidationError


# Beispiel 2: wie positive_at(1) bei testfun(a,b,c), nur b muss > 0
class TestFunInput(BaseModel):
    a: int
    b: int = Field(gt=0)
    c: int


def testfun_checked(a: int, b: int, c: int):
    data = TestFunInput(a=a, b=b, c=c)  # validiert b > 0
    return data.a, data.b, data.c


print(testfun_checked(-1, 1, -1))
# print(testfun_checked(-1, -1, -1))  # ValidationError


Note: you may need to restart the kernel to use updated packages.
4.0
(-1, 1, -1)


c:\Users\cank2\Desktop\Softwaredesigne-KAL\.venv\Scripts\python.exe: No module named pip


In [4]:
# Exercise E.3 (Define log level via environment variable)

import os
import logging

# If myloglevel=DEBUG (or INFO/WARNING/ERROR/CRITICAL) is set, use it.
# Otherwise, use default INFO.
level_name = os.getenv("myloglevel", "INFO").upper()
level = getattr(logging, level_name, logging.INFO)

logging.basicConfig(
    format="%(asctime)s : %(levelname)s : %(message)s",
    level=level,
)

numberlist = [-2, -1, 0, 1, 2, "a", 1 / 4]

for number in numberlist:
    try:
        logging.debug(f"Working on number {number}")
        inverse = 1.0 / number
        logging.info(f"inverse({number}) = {inverse}")
    except ZeroDivisionError as e:
        logging.error(f"Tried to divide by zero, error is {e}")
    except TypeError:
        logging.warning("The list does not only contain numbers")


2026-02-16 16:23:28,714 : INFO : inverse(-2) = -0.5
2026-02-16 16:23:28,715 : INFO : inverse(-1) = -1.0
2026-02-16 16:23:28,715 : ERROR : Tried to divide by zero, error is float division by zero
2026-02-16 16:23:28,716 : INFO : inverse(1) = 1.0
2026-02-16 16:23:28,716 : INFO : inverse(2) = 0.5
2026-02-16 16:23:28,718 : WARNING : The list does not only contain numbers
2026-02-16 16:23:28,718 : INFO : inverse(0.25) = 4.0


In [5]:
# Exercise E.4 (Pathlib)

from pathlib import Path
from collections import Counter

# 1) Create directory tmp in current working directory (if not existing)
tmp_dir = Path.cwd() / "tmp"
tmp_dir.mkdir(exist_ok=True)

# Alternative with try/except:
# try:
#     tmp_dir.mkdir()
# except FileExistsError:
#     pass

# 2) Create files inside tmp
files_to_create = ["test.txt", "file.txt", "README.md", "random.py"]
for name in files_to_create:
    (tmp_dir / name).write_text(f"Created {name}\n", encoding="utf-8")

# 3) Count files per suffix
suffix_counter = Counter(p.suffix for p in tmp_dir.iterdir() if p.is_file())
print(suffix_counter)  # expected: Counter({'.txt': 2, '.md': 1, '.py': 1})

# 4) Find last modified file
all_files = [p for p in tmp_dir.iterdir() if p.is_file()]
last_modified = max(all_files, key=lambda p: p.stat().st_mtime)
print("Last modified file:", last_modified.name)

# 5) Cleanup: delete files and directory
for p in all_files:
    p.unlink()
tmp_dir.rmdir()

print("Cleanup done.")


Counter({'.txt': 2, '.py': 1, '.md': 1})
Last modified file: file.txt
Cleanup done.
